<a href="https://colab.research.google.com/github/casualcomputer/ml-stats-models/blob/main/business_process_modeling_in_R_cpu_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Markov Modeling - Sampling from continous markov chain


## 1. No parallel programming in CPU

In [4]:
library(parallel)
library(ggplot2)

set.seed(123)  # For reproducibility

# Define the rate (generator) matrix
Q <- as.matrix(rbind(
  c(0, 0.25, 0, 0.25),
  c(0.166, 0, 0.166, 0.166),
  c(0, 0.25, 0, 0.25),
  c(0, 0, 0, 0)
))

# Define the state space
states <- 1:nrow(Q)

# Function to choose the next state based on current state and rate matrix
next_state <- function(current_state, Q) {
  rates <- Q[current_state, ]
  if (sum(rates) == 0) {
    return(current_state)  # Stay in the current state if no transitions are possible
  }
  probs <- rates / sum(rates)
  return(sample(states, size = 1, prob = probs))
}

# Function to determine the holding time based on the current state
holding_time <- function(current_state, Q) {
  rate <- sum(Q[current_state, ])
  if (rate == 0) {
    return(Inf)  # Infinite holding time if no transitions are possible
  }
  return(rexp(1, rate = rate))
}

# Function to simulate a single Markov chain sequence
simulate_markov_chain <- function(Q, n_transitions, initial_state) {
  current_state <- initial_state
  current_time <- 0
  sequence_of_states <- c(current_state)
  transition_times <- c(current_time)

  for (i in 1:n_transitions) {
    ht <- holding_time(current_state, Q)
    if (ht == Inf) {
      break  # Stop if holding time is infinite
    }
    current_time <- current_time + ht
    current_state <- next_state(current_state, Q)

    sequence_of_states <- c(sequence_of_states, current_state)
    transition_times <- c(transition_times, current_time)
  }

  return(data.frame(Time = transition_times, State = sequence_of_states))
}

# Parameters
n_simulations <- 1000000
n_transitions <- 100
initial_state <- 1

# Start time
start_time <- Sys.time()

# List to store results of all simulations
all_simulations <- list()

# Generate multiple series
for (i in 1:n_simulations) {
  sim_result <- simulate_markov_chain(Q, n_transitions, initial_state)
  sim_result$Simulation <- i  # Add a column to identify the simulation
  all_simulations[[i]] <- sim_result
}

# Combine all simulations into a single data frame
combined_results <- do.call(rbind, all_simulations)

# End time
end_time <- Sys.time()

# Calculate and print runtime
run_time <- end_time - start_time
print(paste("Start time:", start_time))
print(paste("End time:", end_time))
print(paste("Run time:", run_time))

# Print the combined results summary
summary(combined_results)

# Plot the state sequences for all simulations
# library(ggplot2)
# ggplot(combined_results, aes(x = Time, y = State, color = factor(Simulation))) +
#   geom_step() +
#   labs(title = "Multiple Continuous-Time Markov Chain Simulations",
#        x = "Time",
#        y = "State",
#        color = "Simulation") +
#   theme_minimal()

[1] "Start time: 2024-06-16 03:39:34.692605"
[1] "End time: 2024-06-16 03:45:06.328434"
[1] "Run time: 5.52726381619771"


      Time            State         Simulation     
 Min.   : 0.000   Min.   :1.000   Min.   :      1  
 1st Qu.: 0.000   1st Qu.:1.000   1st Qu.: 249877  
 Median : 1.489   Median :2.000   Median : 499818  
 Mean   : 3.237   Mean   :2.308   Mean   : 499911  
 3rd Qu.: 4.745   3rd Qu.:4.000   3rd Qu.: 749926  
 Max.   :63.556   Max.   :4.000   Max.   :1000000  

## 2. Parallel programming in CPU

In [5]:
library(parallel)
library(ggplot2)

set.seed(123)  # For reproducibility

# Start time
start_time <- Sys.time()

# Set up parallel execution
n_cores <- detectCores() - 1  # Use one less than the number of available cores
cl <- makeCluster(n_cores)

# Export necessary objects and functions to the cluster
clusterExport(cl, c("Q", "n_transitions", "initial_state", "states", "next_state", "holding_time", "simulate_markov_chain"))

# Use parLapply to execute the simulation function in parallel
all_simulations <- parLapply(cl, 1:n_simulations, function(x) {
  sim_result <- simulate_markov_chain(Q, n_transitions, initial_state)
  sim_result$Simulation <- x  # Add a column to identify the simulation
  return(sim_result)
})

# Stop the cluster
stopCluster(cl)

# Combine all simulations into a single data frame
combined_results <- do.call(rbind, all_simulations)

# End time
end_time <- Sys.time()

# Calculate and print runtime
run_time <- end_time - start_time
print(paste("Start time:", start_time))
print(paste("End time:", end_time))
print(paste("Run time:", run_time))

# Print the combined results summary
summary(combined_results)

[1] "Start time: 2024-06-16 03:45:06.812621"
[1] "End time: 2024-06-16 03:50:21.123162"
[1] "Run time: 5.23850901126862"


      Time            State         Simulation     
 Min.   : 0.000   Min.   :1.000   Min.   :      1  
 1st Qu.: 0.000   1st Qu.:1.000   1st Qu.: 250199  
 Median : 1.486   Median :2.000   Median : 499987  
 Mean   : 3.234   Mean   :2.308   Mean   : 499998  
 3rd Qu.: 4.736   3rd Qu.:4.000   3rd Qu.: 749741  
 Max.   :61.780   Max.   :4.000   Max.   :1000000  

## 3. GPU accelerated with Tensorflow - not working

In [18]:
devtools::install_github("rstudio/keras")
#install.packages("keras")
# reticulate: Reticulate: R Interface to Python
# tfruns: Track and Visualize Training Runs
# zeallot: multiple, unpacking, destructuring assignments w/ %<-% operator

rlang      (1.1.3  -> 1.1.4 ) [CRAN]
backports  (1.4.1  -> 1.5.0 ) [CRAN]
rstudioapi (0.15.0 -> 0.16.0) [CRAN]
tidyselect (1.2.0  -> 1.2.1 ) [CRAN]
whisker    (0.4    -> 0.4.1 ) [CRAN]
processx   (3.8.3  -> 3.8.4 ) [CRAN]
fastmap    (1.1.1  -> 1.2.0 ) [CRAN]


Installing 7 packages: rlang, backports, rstudioapi, tidyselect, whisker, processx, fastmap

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpyHF7K8/remotes3a0480451b1/rstudio-keras3-fa07f20/DESCRIPTION’ ... OK
* preparing ‘keras3’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘keras3_1.0.0.9000.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [21]:
install.packages("keras")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [19]:
# Define the rate (generator) matrix
Q <- matrix(c(
  0, 0.25, 0, 0.25,
  0.166, 0, 0.166, 0.166,
  0, 0.25, 0, 0.25,
  0, 0, 0, 0),
  nrow = 4, byrow = TRUE
)

# Convert Q to a TensorFlow constant
Q_tf <- tf$constant(Q, dtype = tf$float32)

# Function to choose the next state based on current state and rate matrix
next_state <- function(current_state, Q_tf) {
  rates <- Q_tf[current_state, ]
  probs <- rates / tf$reduce_sum(rates)
  sample <- tf$random$categorical(tf$log(tf$reshape(probs, shape = c(1, -1))), num_samples = 1)
  return(tf$cast(sample, tf$int32) + 1L)
}

# Function to determine the holding time based on the current state
holding_time <- function(current_state, Q_tf) {
  rate <- tf$reduce_sum(Q_tf[current_state, ])
  ht <- tf$random$exponential(shape = list(), rate = rate)
  return(ht)
}

# Function to simulate a single Markov chain sequence
simulate_markov_chain <- function(Q_tf, n_transitions, initial_state) {
  current_state <- initial_state - 1L  # Adjust for 0-indexing in TensorFlow
  current_time <- 0.0
  sequence_of_states <- list(current_state)
  transition_times <- list(current_time)

  for (i in seq_len(n_transitions)) {
    ht <- holding_time(current_state, Q_tf)
    current_time <- current_time + ht
    current_state <- next_state(current_state, Q_tf)

    sequence_of_states <- c(sequence_of_states, current_state)
    transition_times <- c(transition_times, current_time)
  }

  df <- data.frame(
    Time = as.numeric(unlist(transition_times)),
    State = as.integer(unlist(sequence_of_states)) + 1  # Adjust back to 1-indexing
  )
  return(df)
}

# Parameters
n_simulations <- 1000000
n_transitions <- 100
initial_state <- 1

# Start time
start_time <- Sys.time()

# Generate multiple series
all_simulations <- vector("list", n_simulations)
for (i in seq_len(n_simulations)) {
  all_simulations[[i]] <- simulate_markov_chain(Q_tf, n_transitions, initial_state)
  all_simulations[[i]]$Simulation <- i
}

# Combine all simulations into a single data frame
combined_results <- do.call(rbind, all_simulations)

# End time
end_time <- Sys.time()

# Calculate and print runtime
run_time <- end_time - start_time
print(paste("Start time:", start_time))
print(paste("End time:", end_time))
print(paste("Run time:", run_time))

# Print the combined results summary
summary(combined_results)

# Plot the state sequences for all simulations
# library(ggplot2)
# ggplot(combined_results, aes(x = Time, y = State, color = factor(Simulation))) +
#   geom_step() +
#   labs(title = "Multiple Continuous-Time Markov Chain Simulations",
#        x = "Time",
#        y = "State",
#        color = "Simulation") +
#   theme_minimal()


ERROR: Error: Valid installation of TensorFlow not found.

Python environments searched for 'tensorflow' package:
 /usr/bin/python3.10

Python exception encountered:
 Traceback (most recent call last):
  File "/usr/local/lib/R/site-library/reticulate/python/rpytools/loader.py", line 122, in _find_and_load_hook
    return _run_hook(name, _hook)
  File "/usr/local/lib/R/site-library/reticulate/python/rpytools/loader.py", line 96, in _run_hook
    module = hook()
  File "/usr/local/lib/R/site-library/reticulate/python/rpytools/loader.py", line 120, in _hook
    return _find_and_load(name, import_)
ModuleNotFoundError: No module named 'tensorflow'


You can install TensorFlow using the install_tensorflow() function.



## 3. GPU accelerated with PyTorch: not working

In [ ]:
# Install reticulate package to interface with Python
install.packages("reticulate")
library(reticulate)

# Use reticulate to interface with Python
use_python("/usr/bin/python3")  # Specify the path to your Python interpreter
py_install("torch", pip = TRUE)

# Load the PyTorch library via reticulate
torch <- import("torch")

# Define the rate (generator) matrix
Q <- matrix(c(
  0, 0.25, 0, 0.25,
  0.166, 0, 0.166, 0.166,
  0, 0.25, 0, 0.25,
  0, 0, 0, 0),
  nrow = 4, byrow = TRUE
)

# Convert Q to a PyTorch tensor and move to GPU
Q_torch <- torch$from_numpy(Q)$float()$cuda()

# Function to choose the next state based on current state and rate matrix
next_state <- function(current_state, Q_torch) {
  rates <- Q_torch[current_state, ]
  probs <- rates / rates$sum()
  sample <- torch$multinomial(probs, 1)
  return(sample$item())
}

# Function to determine the holding time based on the current state
holding_time <- function(current_state, Q_torch) {
  rate <- Q_torch[current_state, ]$sum()
  ht <- torch$distributions$Exponential(rate)$sample()
  return(ht$item())
}

# Function to simulate a single Markov chain sequence
simulate_markov_chain <- function(Q_torch, n_transitions, initial_state) {
  current_state <- initial_state - 1  # Adjust for 0-indexing in PyTorch
  current_time <- 0.0
  sequence_of_states <- list(current_state)
  transition_times <- list(current_time)

  for (i in seq_len(n_transitions)) {
    ht <- holding_time(current_state, Q_torch)
    current_time <- current_time + ht
    current_state <- next_state(current_state, Q_torch)

    sequence_of_states <- c(sequence_of_states, current_state)
    transition_times <- c(transition_times, current_time)
  }

  df <- data.frame(
    Time = unlist(transition_times),
    State = unlist(sequence_of_states) + 1  # Adjust back to 1-indexing
  )
  return(df)
}

# Parameters
n_simulations <- 10
n_transitions <- 100
initial_state <- 1

# Start time
start_time <- Sys.time()

# Generate multiple series
all_simulations <- vector("list", n_simulations)
for (i in seq_len(n_simulations)) {
  all_simulations[[i]] <- simulate_markov_chain(Q_torch, n_transitions, initial_state)
  all_simulations[[i]]$Simulation <- i
}

# Combine all simulations into a single data frame
combined_results <- do.call(rbind, all_simulations)

# End time
end_time <- Sys.time()

# Calculate and print runtime
run_time <- end_time - start_time
print(paste("Run time:", run_time))

# Print the combined results summary
summary(combined_results)

# Plot the state sequences for all simulations
library(ggplot2)
ggplot(combined_results, aes(x = Time, y = State, color = factor(Simulation))) +
  geom_step() +
  labs(title = "Multiple Continuous-Time Markov Chain Simulations",
       x = "Time",
       y = "State",
       color = "Simulation") +
  theme_minimal()


## Visualization